----
### Adding workers

In [1]:
@show procs()
@show workers()
;

procs() => [1]
workers() => [1]


In [10]:
addprocs(5)

5-element Array{Any,1}:
  7
  8
  9
 10
 11

In [3]:
workers()

5-element Array{Int64,1}:
 2
 3
 4
 5
 6

In [11]:
procs()

6-element Array{Int64,1}:
  1
  7
  8
  9
 10
 11

In [24]:
((a,b) -> a+b)(1,1)

2

In [25]:
f = (a,b) -> a+b
f(1,1)

2

In [27]:
pmap(x->compute(loadfromid(x)), 1:10)

10-element Array{Any,1}:
  7
  8
  9
 10
 11
  8
 10
  7
  9
 11

In [ ]:
@show procs()

In [ ]:
addprocs({"myserver.test.com"})

In [ ]:
n = 10
makedata(i) = rand(1000,i*10)
data = [makedata(i) for i in 1:n];

In [ ]:
map(size,data)'

In [ ]:
map(svdvals,data)
@time map(svdvals,data);

In [ ]:
pmap(svdvals,data)
@time pmap(svdvals,data);

----
### Manual scheduling

In [28]:
ref = @spawn svdvals(rand(1000,1000))

RemoteRef(7,1,47)

In [29]:
fetch(ref)

1000-element Array{Float64,1}:
 499.923    
  18.1002   
  18.0527   
  17.8631   
  17.8041   
  17.7606   
  17.7374   
  17.6571   
  17.5685   
  17.5353   
  17.4853   
  17.4503   
  17.4118   
   ⋮        
   0.158887 
   0.145721 
   0.136169 
   0.129068 
   0.119272 
   0.0882336
   0.0812822
   0.064076 
   0.054219 
   0.0441545
   0.0311214
   0.0133422

In [30]:
refs = [@spawn svdvals(rand(100,100)) for i in 1:10]

10-element Array{Any,1}:
 RemoteRef(8,1,49) 
 RemoteRef(9,1,50) 
 RemoteRef(10,1,51)
 RemoteRef(11,1,52)
 RemoteRef(7,1,53) 
 RemoteRef(8,1,54) 
 RemoteRef(9,1,55) 
 RemoteRef(10,1,56)
 RemoteRef(11,1,57)
 RemoteRef(7,1,58) 

In [31]:
map(fetch,refs)

10-element Array{Any,1}:
 [50.3365,5.47472,5.36955,5.22753,5.10941,5.08641,4.99248,4.75055,4.69099,4.67889  …  0.441236,0.4034,0.315258,0.287224,0.192707,0.147904,0.134402,0.0989697,0.023843,0.00592803]  
 [50.2771,5.45235,5.36251,5.28037,5.20253,5.10001,4.97308,4.89468,4.82153,4.75513  …  0.416831,0.3647,0.332919,0.279563,0.223766,0.197548,0.142849,0.103686,0.0524814,0.0375725]   
 [49.9718,5.55091,5.39783,5.30926,5.11027,5.03301,5.00418,4.92975,4.85348,4.72352  …  0.40538,0.340506,0.299965,0.276145,0.214589,0.166687,0.127433,0.122173,0.0646829,0.00292012] 
 [49.476,5.51095,5.40403,5.3149,5.18318,4.99017,4.97137,4.90749,4.80682,4.71799  …  0.418037,0.407766,0.348786,0.311434,0.256461,0.181809,0.166338,0.096219,0.076406,0.0340132]    
 [50.3707,5.62929,5.44392,5.31686,5.24275,5.1677,4.95784,4.85054,4.75139,4.71503  …  0.448443,0.408019,0.368464,0.293881,0.252609,0.170489,0.139245,0.122907,0.0845574,0.0142582]  
 [50.0571,5.57256,5.27454,5.21227,5.16191,4.99658,4.94638,4.9004,4.75934,4.

In [32]:
@fetch svdvals(rand(10,10))'

1x10 Array{Float64,2}:
 4.8328  1.51182  1.29296  1.14295  …  0.394631  0.154713  0.000162984

In [34]:
@fetchfrom 7 svdvals(rand(10,10))'

1x10 Array{Float64,2}:
 5.05771  1.56697  1.23493  1.224  0.971407  …  0.516634  0.327495  0.199651

In [36]:
@fetchfrom 8 myid()

8

In [ ]:
pmap(x->myid(), 1:10)'

In [ ]:
r = cell(10)
for i = 1:10
    r[i] = @fetch svdvals(rand(10*1,10))
end
r

In [ ]:
r = cell(10)
@sync for i = 1:10
    @async r[i] = @fetch svdvals(rand(10*1,10))
end
r

In [ ]:
data = rand(1000,100_000)
@time serialize(IOBuffer(),data);

In [38]:
@edit pmap(sum, 1:10)

Unknown editor: no line number information passed.
The method is defined at line 1326.


------
### @parallel

In [ ]:
nheads = @parallel (+) for i=1:200000000
  int(randbool())
end

------
### Distributed Memory


In [39]:
data = dzeros(2,3,4)

2x3x4 DArray{Float64,3,Array{Float64,3}}:
[:, :, 1] =
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 4] =
 0.0  0.0  0.0
 0.0  0.0  0.0

In [40]:
localpart(data)

0x0x0 Array{Float64,3}

In [43]:
@fetchfrom 7 localpart(data)

2x2x2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 2] =
 0.0  0.0
 0.0  0.0

in Julia 0.4 moved to a package: https://github.com/JuliaParallel/DistributedArrays.jl

------
### Shared Memory


In [44]:
a = SharedArray(Int, (3,4))

3x4 SharedArray{Int64,2}:
 0  0  0  0
 0  0  0  0
 0  0  0  0

In [47]:
a = SharedArray(Int, (3,4), init = S -> S[localindexes(S)] = myid())

3x4 SharedArray{Int64,2}:
 1  7   9  10
 1  8   9  11
 7  8  10  11

In [46]:
@fetchfrom 8 a[1]

0

------
### Packages for parallel programming

* Low level: https://github.com/JuliaParallel/MPI.jl
* Communicating Sequential Processes: https://github.com/shashi/CSP.jl
* FunctionalData: https://github.com/rened/FunctionalData.jl

In [48]:
using FunctionalData

In [54]:
data = [1 2 3; 1 2 3]

2x3 Array{Int64,2}:
 1  2  3
 1  2  3

In [57]:
@everywhere addone(x) = x+1
map(data, addone)

2x3 Array{Int64,2}:
 2  3  4
 2  3  4

In [58]:
pmap(data, addone)

2x3 Array{Int64,2}:
 2  3  4
 2  3  4

In [59]:
shmap(data, addone)

2x3 SharedArray{Int64,2}:
 2  3  4
 2  3  4

In [60]:
lmap(data, addone)

2x3 Array{Int64,2}:
 2  3  4
 2  3  4

In [61]:
hmap(data, addone)

2x3 Array{Int64,2}:
 2  3  4
 2  3  4

In [64]:
map({1,"test",rand(2,3)}, typeof)

3-element Array{Any,1}:
 Int64           
 ASCIIString     
 Array{Float64,2}

In [65]:
# take care, Base.map and pmap return different results:
Base.map(addone, data)

2x3 Array{Int64,2}:
 2  3  4
 2  3  4

In [66]:
Base.pmap(addone, data)

6-element Array{Any,1}:
 2
 2
 3
 3
 4
 4